# Continuous Pendulum Control
Zero reward is the best condition for the pendulum control

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from mvp.env_pendulum import PendulumEnv

from itertools import count
import torch
import gym
from gym.envs.registration import register

import matplotlib
import matplotlib.pyplot as plt
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
plt.ion()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pendulum Control Gym

## Description
The inverted pendulum swingup problem is a classic problem in control theory. The system consists of a pendulum attached at one end to a fixed point, with the other end being free. The pendulum starts in a random position, and the goal is to apply torque to the free end to swing it into an upright position, where its center of gravity is right above the fixed point.

### Pendulum Coordinate System

- **x-y**: Cartesian coordinates of the pendulum’s end in meters.
- **theta**: Angle in radians.
- **tau**: Torque in N·m, defined as positive counter-clockwise.

### Action Space
The action is an ndarray with shape `(1,)` representing the torque applied to the free end of the pendulum.

| Num | Action | Min | Max |
|-----|--------|-----|-----|
| 0   | Torque | -2.0| 2.0 |

### Observation Space
The observation is an ndarray with shape `(3,)` representing the x-y coordinates of the pendulum’s free end and its angular velocity.

| Num | Observation      | Min | Max |
|-----|------------------|-----|-----|
| 0   | x = cos(theta)   | -1.0| 1.0 |
| 1   | y = sin(theta)   | -1.0| 1.0 |
| 2   | Angular Velocity | -8.0| 8.0 |

### Rewards
The reward function is defined as:

$r = -(theta^2 + 0.1 * theta_dt^2 + 0.001 * torque^2)$

where `theta` is the pendulum’s angle normalized between `[-pi, pi]` (with 0 being in the upright position). The minimum reward that can be obtained is `-(pi^2 + 0.1 * 8^2 + 0.001 * 2^2) = -16.2736044`, while the maximum reward is zero (pendulum is upright with zero velocity and no torque applied).

In [2]:
register(
    id='Pendulum-v0',
    entry_point='mvp.env_pendulum:PendulumEnv',
    max_episode_steps=1000)
env = gym.make('Pendulum-v0')

/Users/kevinb/anaconda3/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment Pendulum-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [3]:
env.action_space

Box(-2.0, 2.0, (1,), float32)

In [4]:
env.reward_range

(-inf, inf)

In [5]:
env.observation_space

Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)

# Evaluation of Model

In [6]:
import numpy as np
from mvp.ppo_continuous import ActorCritic

path = os.path.join(os.getcwd(), "..", "mvp", "params", "pendulum_ppo_continuous.pth")

env = PendulumEnv(render_mode="human")
n_actions = env.action_space.shape[0]
state, info = env.reset()
n_observations = len(state)
num_eval_episodes = 10

model = ActorCritic(n_observations, n_actions).to(device)
model.load_state_dict(torch.load(path, map_location=device))
model.eval()

for i_episode in range(num_eval_episodes):
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        env.render()
        # Get action from the model (assuming it outputs action_mean, action_std, and value)
        with torch.no_grad():
            action_mean, _, _ = model(state)
        
        # Take the mean action (no sampling here for deterministic behavior)
        action = action_mean.cpu().numpy()[0]
        print(action)

        observation, reward, terminated, truncated, _ = env.step(action)
        if terminated or truncated:
            print(f"Episode finished after {t+1} timesteps")
            break

        state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
        
env.close()

/Users/kevinb/anaconda3/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment Pendulum-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


[2.4207256]
[2.446133]
[2.2714994]
[1.8940108]
[1.3232177]
[0.55958474]
[-0.26190308]
[-0.9278264]
[-1.3315694]
[-1.5126436]
[-1.548559]
[-1.4980936]
[-1.4082962]
[-1.3291913]
[-1.3071458]
[-1.3675156]
[-1.5068123]
[-1.695585]
[-1.888982]
[-2.0432642]
[-2.138118]
[-2.1743402]
[-2.1471665]
[-2.0604918]
[-1.9223309]
[-1.7528021]
[-1.5728025]
[-1.3908703]
[-1.213364]
[-1.0447677]
[-0.8880079]
[-0.7447523]
[-0.61569136]
[-0.5007874]
[-0.39949605]
[-0.31094626]
[-0.23408435]
[-0.16778274]
[-0.11091457]
[-0.06240374]
[-0.02125367]
[0.01343834]
[0.04247668]
[0.06656948]
[0.08633515]
[0.1023097]
[0.11495648]
[0.1246736]
[0.13180313]
[0.13663837]
[0.13943]
[0.14039281]
[0.13971016]
[0.13753851]
[0.1340114]
[0.12924245]
[0.1233283]
[0.11635067]
[0.10837884]
[0.09947072]
[0.08967483]
[0.07903172]
[0.06757395]
[0.05532838]
[0.04231625]
[0.02855388]
[0.01405319]
[-0.00117754]
[-0.01713327]
[-0.033812]
[-0.05121448]
[-0.06934341]
[-0.0882039]
[-0.10780291]
[-0.12814893]
[-0.14925222]
[-0.1711241]
[-

KeyboardInterrupt: 